# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker
!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

## Sagemaker Prep

### Session

Configure SageMaker

In [6]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

In [7]:
from mlflow import sagemaker

In [8]:
sagemaker.deploy(
    'test-pyfunc-all-models',
    's3://deep-mlflow-artifact/16/1540c9852a7c47eab93beb012b8d7749/artifacts/pyfunc_models_all',
    execution_role_arn=SAGEMAKER_ROLE_ARN,
    image_url="961104659532.dkr.ecr.us-east-1.amazonaws.com/mlflow-pyfunc:latest",
    region_name="us-east-1",
    instance_type="ml.g4dn.xlarge",
    synchronous=False,
    archive=True,
)

2021/10/26 11:10:57 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2021/10/26 11:10:58 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2021/10/26 11:10:59 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-1-961104659532` already exists. Skipping creation.
2021/10/26 11:15:14 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': 'BSBZ6SJBDXZCRGXX', 'HostId': 'cLvNgOUnCPJriUAD6zmuL8c7KeTl6Xqeewrodktd3yVz6XNBnH/IeTOrY+x0HU5muuSVmN5sLrg=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'cLvNgOUnCPJriUAD6zmuL8c7KeTl6Xqeewrodktd3yVz6XNBnH/IeTOrY+x0HU5muuSVmN5sLrg=', 'x-amz-request-id': 'BSBZ6SJBDXZCRGXX', 'date': 'Tue, 26 Oct 2021 09:15:14 GMT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}
2021/10/26 11:15:14 INFO mlflow.sagemaker: Creating new endpoint with name: test-pyfunc-all-models ...
2021/10/26 11:15:14 INFO mlflow.sagemaker: Created model with arn: arn:aws:sagemaker:us

In [9]:
test_data = pd.read_excel(os.path.join('..', '..', '..', '..', 'feedback_output.xlsx'))
test_data = test_data[['Entry']].rename(columns={'Entry':'excerpt'})
test_data = test_data[test_data.excerpt.apply(lambda x: 'NONE' != x.upper())]

In [11]:
import boto3
import timeit

start = timeit.default_timer()
client = boto3.session.Session().client("sagemaker-runtime", region_name='us-east-1')

data = test_data
input_json = data.to_json(orient="split")

response = client.invoke_endpoint(
    EndpointName='test-pyfunc-all-models',
    Body=input_json,
    ContentType="application/json; format=pandas-split",
)
output = response["Body"].read().decode("ascii")
end = timeit.default_timer()

In [17]:
from ast import literal_eval
output = literal_eval(output)


In [19]:
preds = output[0]
thresholds = output[1]

In [101]:
def flatten(t):
    return [item for sublist in t for item in sublist]

import numpy as np

multilabel_columns = [
    'sectors', 
    'pillars_2d',
    'pillars_1d',
    'subpillars_2d', 
    'subpillars_1d', 
    'demographic_groups', 
    'affected_groups', 
    'specific_needs_groups'
    ]

no_subpillar_columns = [
    'sectors',
    'demographic_groups', 
    'affected_groups', 
    'specific_needs_groups',
    'subpillars_2d', 
    'subpillars_1d',
    ]

all_columns = [
    'sectors', 
    'subpillars_2d', 
    'subpillars_2d_postprocessed',
    'subpillars_1d', 
    'subpillars_1d_postprocessed',
    'demographic_groups', 
    'affected_groups', 
    'specific_needs_groups',
    'severity'
    ]

def postprocess_subpillars (ratios_pillars, ratios_subpillars, return_at_least_one=True):
    """
    

    """
    
    results_subpillars = []

    ratios_subpillars_changed = {name: {} for name, _ in ratios_pillars.items()}
    for column_name, ratio in ratios_subpillars.items():
        split_column = column_name.split('->')
        ratios_subpillars_changed[split_column[0]].update({
            split_column[1]: ratio
        })

    positive_pillars = [
        column_name for column_name, ratio in ratios_pillars.items() if ratio >= 1
        ]
    if len (positive_pillars) == 0 and return_at_least_one:
        positive_pillars = [
        column_name for column_name, ratio in ratios_pillars.items() \
            if ratio == max(list(ratios_pillars.values()))
        ]

    if len (positive_pillars) == 0:
        return []
    
    for column_tmp in positive_pillars:
        dict_results_column = ratios_subpillars_changed[column_tmp]
        preds_column_tmp = [
            f"{column_tmp}->{subtag}" for subtag, value in dict_results_column.items() if value >=1
        ]
        if len(preds_column_tmp)==0:
            preds_column_tmp = [
            subtag for subtag, value in dict_results_column.items() \
                if value == max(list(dict_results_column.values()))
        ]
        results_subpillars.append(preds_column_tmp)
        
    return flatten(results_subpillars)

def get_predictions(test_probas, thresholds_dict):  
    """
    test_probas structure example: {
        'sectors':[
            {'Nutrition': 0.032076582, 'Shelter': 0.06674846}, 
            {'Cross': 0.21885818,'Education': 0.07529669}
        ],
        'demographic_groups':[
            {'Children/Youth Female (5 to 17 years old)': 0.47860646, 'Children/Youth Male (5 to 17 years old)': 0.42560646},
            {'Children/Youth Male (5 to 17 years old)': 0.47860646, 'Infants/Toddlers (<5 years old)': 0.85}
        ],
        .
        .
        .
    }
    
    thresholds_dict structure example: {
        'sectors':{
            'Agriculture': 0.2,
            'Cross': 0.02,
            .
            .
        },
        'subpillars_2d':{
            'Humanitarian Conditions->Physical And Mental Well Being': 0.7,
            .
            .
        },
        .
        .     
    }
    
    First iteration:
    - create dict which has the same structure as 'test_probas': 
    - contains ratio probability of output divided by the threshold
    
    Second iteration:
    - keep ratios superior to 1 except:
        - for subpillars_2d: when no ratio is superior to 1 but there is at least one prediction for sectors
        - for severity (no threshold, just keep max if there is 'Humanitarian Conditions' in secondary tags outputs)
    """

    #create dict of ratio between probability of output and threshold
    ratio_proba_threshold = {}
    for column in multilabel_columns:
        preds_column = test_probas[column]
        dict_keys = list(thresholds_dict[column].keys())
        nb_entries = len([i for i in test_probas['sectors'] if i])

        returned_values_column = []
        for preds_sent in preds_column:
            dict_entry = {key:preds_sent[key]/thresholds_dict[column][key] for key in dict_keys }
            returned_values_column.append(dict_entry)
        ratio_proba_threshold[column] = returned_values_column

    predictions = {column:[] for column in all_columns}
    for entry_nb in range (nb_entries):

        # get the entries where the ratio is superior to 1 and put them in a dict {prediction:probability}
        for column in no_subpillar_columns:
            preds_column = ratio_proba_threshold[column][entry_nb]
            preds_entry = [
                sub_tag for sub_tag in list(preds_column.keys()) if ratio_proba_threshold[column][entry_nb][sub_tag]>1
            ]

            #postprocessing to keep only cross if more than one prediction
            if column=='sectors' and len(preds_entry)>1:
                preds_entry.append('Cross')

            predictions[column].append(list(np.unique(preds_entry)))

        preds_2d = postprocess_subpillars(
            ratio_proba_threshold['pillars_2d'][entry_nb],
            ratio_proba_threshold['subpillars_2d'][entry_nb],
            True
            )

        preds_1d = postprocess_subpillars(
            ratio_proba_threshold['pillars_1d'][entry_nb],
            ratio_proba_threshold['subpillars_1d'][entry_nb],
            False
            )

        predictions['subpillars_2d_postprocessed'].append(preds_2d)
        predictions['subpillars_1d_postprocessed'].append(preds_1d)

        #postprocess 'subpillars_2d'
        if len(predictions['sectors'][entry_nb])>0 and len(predictions['subpillars_2d'][entry_nb])==0:
            predictions['subpillars_2d'][entry_nb] = [
                sub_tag for sub_tag in list(preds_column.keys()) if\
                        test_probas[column][entry_nb][sub_tag] == max(list(test_probas[column][entry_nb].values()))
            ]

        if len(predictions['sectors'][entry_nb])==0 and len(predictions['subpillars_2d'][entry_nb])>0:
            predictions['subpillars_2d'][entry_nb] = []
            
        #severity  predictions and output
        if 'Humanitarian Conditions' in str(predictions['subpillars_2d'][entry_nb]):
            pred_severity = [
                sub_tag for sub_tag in list(test_probas['severity'][entry_nb].keys()) if\
                test_probas['severity'][entry_nb][sub_tag] == max(list(test_probas['severity'][entry_nb].values()))
            ]

            predictions['severity'].append(pred_severity)
        else:
            predictions['severity'].append([])
            
    return predictions


In [102]:
final_preds = get_predictions(preds, thresholds)

In [ ]:
from sklearn import metrics

def get_flat_matrix (column_of_columns, tag_to_id, nb_subtags):
    matrix = [[
        1 if tag_to_id[i] in column else 0 for i in range (nb_subtags)
    ] for column in column_of_columns]
    return flatten(matrix)

def assess_performance (preds, groundtruth, subtags):
    nb_subtags = len(subtags)
    tag_to_id = {i:subtags[i] for i in range (nb_subtags)}
    groundtruth_col = get_flat_matrix( groundtruth, tag_to_id, nb_subtags)
    preds_col = get_flat_matrix( preds, tag_to_id, nb_subtags)
    
    results = {
        'precision': metrics.precision_score(groundtruth_col, preds_col, average='macro'),
        'recall': metrics.recall_score(groundtruth_col, preds_col, average='macro'),
        'f1': metrics.fbeta_score(groundtruth_col, preds_col, 0.8, average='macro'),
    }
